In [14]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# Assuming the functions are HTTP endpoints
function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/download_files'
function2_url = 'https://europe-west3-lumpito.cloudfunctions.net/extract_table'
function3_url = 'https://europe-west3-lumpito.cloudfunctions.net/standardize_data'
#numbers 9 to 100 in the company_ids list

company_ids = list(range(101, 201))
import time

# Constants for retry policy
MAX_RETRIES = 3
BACKOFF_FACTOR = 1.5

def call_function(url, json, session):
    for attempt in range(MAX_RETRIES):
        response = session.post(url, json=json)
        if response.status_code == 200:
            return response.json()  # Assuming the function returns JSON
        else:
            time.sleep(BACKOFF_FACTOR ** attempt)  # Exponential backoff
    return None  # All retries failed

def call_function_in_sequence(company_id):
    session = requests.Session()  # Reuse connections within the function
    json_data = {"company_id": company_id}
    json_one = {"company_id": company_id, "documents": "gs"}

    # First function call with retry
    result1 = call_function(function1_url, json_one, session)
    if not result1:
        return company_id, False, 'Function 1 failed'

    # Second function call with retry
    result2 = call_function(function2_url, json_data, session)
    if not result2:
        return company_id, False, 'Function 2 failed'

    # Third function call with retry
    result3 = call_function(function3_url, json_data, session)
    if not result3:
        return company_id, False, 'Function 3 failed'

    return company_id, True, 'Success'

# The rest of your code remains the same

# The number of workers can be adjusted based on your system and network capabilities
workers = 15 

with ThreadPoolExecutor(max_workers=workers) as executor:
    # Dictionary to keep track of futures for each company_id
    future_to_company_id = {executor.submit(call_function_in_sequence, cid): cid for cid in company_ids}

    for future in as_completed(future_to_company_id):
        company_id = future_to_company_id[future]
        try:
            _, success, message = future.result()
            if success:
                print(f"Company ID {company_id}: All functions succeeded")
            else:
                print(f"Company ID {company_id}: {message}")
        except Exception as exc:
            print(f"Company ID {company_id} generated an exception: {exc}")

Company ID 115: Function 1 failed
Company ID 102: Function 1 failed
Company ID 112: Function 1 failed
Company ID 101: Function 1 failed
Company ID 107: Function 1 failed
Company ID 105: Function 1 failed
Company ID 108: Function 1 failed
Company ID 110: Function 1 failed
Company ID 116: Function 1 failed
Company ID 118: Function 1 failed
Company ID 119: Function 1 failed
Company ID 122: Function 1 failed
Company ID 125: Function 1 failed
Company ID 126: Function 1 failed
Company ID 114: All functions succeeded
Company ID 104: All functions succeeded
Company ID 124: Function 1 failed
Company ID 106: All functions succeeded
Company ID 103: All functions succeeded
Company ID 128: Function 1 failed
Company ID 135: Function 1 failed
Company ID 134: Function 1 failed
Company ID 136: Function 1 failed
Company ID 137: Function 1 failed
Company ID 132: All functions succeeded
Company ID 131: All functions succeeded
Company ID 129: All functions succeeded
Company ID 130: All functions succeeded


In [38]:
session = requests.Session() 
json={"company_id": 14, "documents": "gs"}
# json={"company_id": 24}
function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/download_files'
response1 = session.post(function1_url, json=json)
# function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/standardize_data'
# response1 = session.post(function1_url, json=json)
print(response1.text)

500 Internal Server Error: The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.
